# Import Tweets

In [1]:
# Import all libraries 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
topic_tweets = pd.read_csv("topic_modelling_output.csv")
sentiment_tweets = pd.read_csv("sentiment_analysis_output.csv")

In [3]:
compound = sentiment_tweets['compound']
trump_tweets = topic_tweets.join(compound)

In [4]:
trump_tweets['tweet_datetime'] = pd.to_datetime(trump_tweets['tweet_datetime'])

In [5]:
trump_tweets

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,topic,compound
0,9.353400e+17,thank rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,1,0.4199
1,8.997980e+17,join live fort myer arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2,0.2960
2,8.939700e+17,thank nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,6,0.4199
3,8.819770e+17,thank shawn steel nice word,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,3,0.6486
4,8.778460e+17,great night iowa special people thank,56446,8039,2017-06-22 11:11:00,2017-06-22 19:11:00,2017-06-22,19:11:00,19,2017,6,6,0.8622
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,9.990960e+17,person placed early campaign spy put previous ...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,3,0.6792
2750,9.874600e+17,general michael flynn life totally destroyed s...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,3,-0.6689
2751,9.870960e+17,thought prayer condolence family friend collea...,62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,2,0.6900
2752,9.863570e+17,today court decision mean congress must close ...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,2,-0.9493


# Import Stock Prices

In [6]:
stock_price = pd.read_csv("stock_price_cleaned.csv")

In [7]:
stock_price.rename(columns={"DATE_TIME_M": "datetime", "PRICE": "price"}, inplace=True)
stock_price['datetime']= pd.to_datetime(stock_price['datetime'])
# stock_price['date'] = stock_price['datetime'].apply(lambda x: x.date())

In [8]:
stock_price

,datetime,price
0,2017-01-20 04:00:00,226.5000
1,2017-01-20 04:01:00,226.4975
2,2017-01-20 04:02:00,226.4950
3,2017-01-20 04:03:00,226.4925
4,2017-01-20 04:04:00,226.4900
...,...,...
683266,2018-12-31 19:56:00,250.0600
683267,2018-12-31 19:57:00,250.0700
683268,2018-12-31 19:58:00,250.1000
683269,2018-12-31 19:59:00,250.0700


# Get End of Day (8pm) Stock Prices

In [9]:
# end_of_day_prices = stock_price[stock_price['datetime'].apply(lambda x: x.time()) == datetime.time(20)]
# end_of_day_prices['datetime'] = end_of_day_prices['datetime'].apply(lambda x: x.date()) 
# end_of_day_prices.rename(columns={"price": "eod_price", "datetime": "eod_date"}, inplace=True)

# Join Stock Prices to EOD Price

In [10]:
# stock_price = stock_price.merge(end_of_day_prices, left_on='date', right_on='eod_date')
# stock_price.drop(columns=['date_x', 'date_y', 'eod_date'], inplace=True)

# Get Stock Price Difference X mins after

In [11]:
stock_price

,datetime,price
0,2017-01-20 04:00:00,226.5000
1,2017-01-20 04:01:00,226.4975
2,2017-01-20 04:02:00,226.4950
3,2017-01-20 04:03:00,226.4925
4,2017-01-20 04:04:00,226.4900
...,...,...
683266,2018-12-31 19:56:00,250.0600
683267,2018-12-31 19:57:00,250.0700
683268,2018-12-31 19:58:00,250.1000
683269,2018-12-31 19:59:00,250.0700


In [12]:
x_mins = 60

In [13]:
stock_price[f"datetime_{x_mins}mins_before"] = stock_price["datetime"] - pd.Timedelta(f"{x_mins} min")

In [14]:
stock_price_merged = stock_price.merge(stock_price, how='inner', left_on=[f"datetime_{x_mins}mins_before"], right_on=["datetime"])
stock_price_merged.drop(columns=["datetime_y", f"datetime_{x_mins}mins_before_y"], inplace=True)

In [15]:
stock_price_merged

,datetime_x,price_x,datetime_60mins_before_x,price_y
0,2017-01-20 05:00:00,226.563333,2017-01-20 04:00:00,226.5000
1,2017-01-20 05:01:00,226.556667,2017-01-20 04:01:00,226.4975
2,2017-01-20 05:02:00,226.550000,2017-01-20 04:02:00,226.4950
3,2017-01-20 05:03:00,226.560000,2017-01-20 04:03:00,226.4925
4,2017-01-20 05:04:00,226.561667,2017-01-20 04:04:00,226.4900
...,...,...,...,...
640606,2018-12-31 19:56:00,250.060000,2018-12-31 18:56:00,249.9900
640607,2018-12-31 19:57:00,250.070000,2018-12-31 18:57:00,249.9800
640608,2018-12-31 19:58:00,250.100000,2018-12-31 18:58:00,250.0000
640609,2018-12-31 19:59:00,250.070000,2018-12-31 18:59:00,250.0000


In [16]:
stock_price_merged.rename(columns={"datetime_x": f"datetime_{x_mins}mins_after", "price_x": f"price_{x_mins}mins_after", f"datetime_{x_mins}mins_before_x": "datetime_now", "price_y": "price_now"}, inplace=True)

In [17]:
stock_price_merged

,datetime_60mins_after,price_60mins_after,datetime_now,price_now
0,2017-01-20 05:00:00,226.563333,2017-01-20 04:00:00,226.5000
1,2017-01-20 05:01:00,226.556667,2017-01-20 04:01:00,226.4975
2,2017-01-20 05:02:00,226.550000,2017-01-20 04:02:00,226.4950
3,2017-01-20 05:03:00,226.560000,2017-01-20 04:03:00,226.4925
4,2017-01-20 05:04:00,226.561667,2017-01-20 04:04:00,226.4900
...,...,...,...,...
640606,2018-12-31 19:56:00,250.060000,2018-12-31 18:56:00,249.9900
640607,2018-12-31 19:57:00,250.070000,2018-12-31 18:57:00,249.9800
640608,2018-12-31 19:58:00,250.100000,2018-12-31 18:58:00,250.0000
640609,2018-12-31 19:59:00,250.070000,2018-12-31 18:59:00,250.0000


In [18]:
stock_price_merged[f'{x_mins}mins_price_diff_abs'] = stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']

stock_price_merged[f'{x_mins}mins_price_diff_perc'] = (stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

# stock_price_merged['eod_price_diff_abs'] = stock_price_merged['eod_price'] - stock_price_merged['price_now']

# stock_price_merged['eod_price_diff_perc'] = (stock_price_merged['eod_price'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

In [19]:
stock_price_merged

,datetime_60mins_after,price_60mins_after,datetime_now,price_now,60mins_price_diff_abs,60mins_price_diff_perc
0,2017-01-20 05:00:00,226.563333,2017-01-20 04:00:00,226.5000,0.063333,0.000280
1,2017-01-20 05:01:00,226.556667,2017-01-20 04:01:00,226.4975,0.059167,0.000261
2,2017-01-20 05:02:00,226.550000,2017-01-20 04:02:00,226.4950,0.055000,0.000243
3,2017-01-20 05:03:00,226.560000,2017-01-20 04:03:00,226.4925,0.067500,0.000298
4,2017-01-20 05:04:00,226.561667,2017-01-20 04:04:00,226.4900,0.071667,0.000316
...,...,...,...,...,...,...
640606,2018-12-31 19:56:00,250.060000,2018-12-31 18:56:00,249.9900,0.070000,0.000280
640607,2018-12-31 19:57:00,250.070000,2018-12-31 18:57:00,249.9800,0.090000,0.000360
640608,2018-12-31 19:58:00,250.100000,2018-12-31 18:58:00,250.0000,0.100000,0.000400
640609,2018-12-31 19:59:00,250.070000,2018-12-31 18:59:00,250.0000,0.070000,0.000280


# Merge Tweets and Stock Prices

In [20]:
merged = trump_tweets.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])

merged.drop(columns=["datetime_now"], inplace=True)

In [21]:
merged.shape

(2246, 18)

In [22]:
merged

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,topic,compound,datetime_60mins_after,price_60mins_after,price_now,60mins_price_diff_abs,60mins_price_diff_perc
0,9.353400e+17,thank rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,1,0.4199,2017-11-28 11:50:00,261.485000,261.100000,0.385000,0.001475
1,8.997980e+17,join live fort myer arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2,0.2960,2017-08-22 10:00:00,244.260000,243.670000,0.590000,0.002421
2,8.939700e+17,thank nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,6,0.4199,2017-05-09 08:01:00,239.940000,239.875000,0.065000,0.000271
3,8.819770e+17,thank shawn steel nice word,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,3,0.6486,2017-03-08 05:44:00,237.022857,236.880000,0.142857,0.000603
4,8.787250e+17,make america great,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,2,0.6588,2017-06-25 06:23:00,243.326476,243.320762,0.005714,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,9.990960e+17,person placed early campaign spy put previous ...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,3,0.6792,2018-05-23 10:13:00,271.930000,271.040000,0.890000,0.003284
2242,9.874600e+17,general michael flynn life totally destroyed s...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,3,-0.6689,2018-04-20 19:34:00,267.025000,266.820000,0.205000,0.000768
2243,9.870960e+17,thought prayer condolence family friend collea...,62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,2,0.6900,2018-04-20 07:30:00,269.070000,268.620000,0.450000,0.001675
2244,9.863570e+17,today court decision mean congress must close ...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,2,-0.9493,2018-04-18 06:34:00,270.695000,270.600000,0.095000,0.000351


In [23]:
merged.head()

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,topic,compound,datetime_60mins_after,price_60mins_after,price_now,60mins_price_diff_abs,60mins_price_diff_perc
0,9.353400e+17,thank rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,1,0.4199,2017-11-28 11:50:00,261.485000,261.100000,0.385000,0.001475
1,8.997980e+17,join live fort myer arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2,0.2960,2017-08-22 10:00:00,244.260000,243.670000,0.590000,0.002421
2,8.939700e+17,thank nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,6,0.4199,2017-05-09 08:01:00,239.940000,239.875000,0.065000,0.000271
3,8.819770e+17,thank shawn steel nice word,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,3,0.6486,2017-03-08 05:44:00,237.022857,236.880000,0.142857,0.000603
4,8.787250e+17,make america great,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,2,0.6588,2017-06-25 06:23:00,243.326476,243.320762,0.005714,0.000023


In [24]:
merged.to_csv("tweets_stocks_combined_final_60mins.csv", index=False, header=True)

# Get Array of Stock Prices 30 minutes before, ie. [-5, -4, -3, -2, -1]

In [52]:
def get_array_of_prices_xmin_before(datetime_now, x_min, stock_price_merged):
    earliest_time = (datetime_now - timedelta(minutes = x_min)).time()
    if earliest_time >= datetime.time(4, 0):
        result = []
        for min_interval in range(x_min, 0, -1):
            datetime_query = datetime_now - timedelta(minutes = min_interval)
            price = stock_price_merged[stock_price_merged['datetime_now'] == datetime_query].iloc[0, stock_price_merged.columns.get_loc('price_now')]
            result.append(price)
        print(f"{datetime_now}: {result}")
        return result
        
    return None

In [53]:
merged[f'prev_30mins_prices'] = merged['tweet_datetime'].apply(get_array_of_prices_xmin_before, args=(30, stock_price_merged))

2017-11-28 10:50:00: [260.96, 261.03, 261.01, 261.015, 261.04, 261.01, 261.02, 261.01, 260.955, 260.94, 260.94, 260.97, 260.97900000000004, 261.01, 261.015, 261.03, 261.08, 261.08, 261.075, 261.125, 261.13, 261.17, 261.105, 261.0861, 261.09, 261.15, 261.11, 261.1, 261.09, 261.0825]
2017-08-22 09:00:00: [243.76, 243.79, 243.85, 243.86, 243.81, 243.78, 243.78, 243.75, 243.75, 243.76, 243.75, 243.75, 243.77, 243.73, 243.75, 243.77, 243.78, 243.765, 243.75, 243.69, 243.66, 243.66, 243.64, 243.65, 243.65, 243.68, 243.69, 243.67, 243.65, 243.66]
2017-05-09 07:01:00: [239.73, 239.73, 239.73, 239.73, 239.73, 239.73, 239.7442857142857, 239.75857142857143, 239.77285714285716, 239.78714285714287, 239.80142857142857, 239.8157142857143, 239.83, 239.83, 239.86, 239.88, 239.88, 239.88, 239.88, 239.88, 239.88, 239.88, 239.88, 239.88, 239.91, 239.91, 239.91, 239.9, 239.89, 239.88]
2017-03-08 04:44:00: [236.84, 236.84, 236.84, 236.8533333333333, 236.86666666666667, 236.88, 236.88, 236.88, 236.88, 236.88

2018-04-01 10:07:00: [262.7880020833333, 262.7879041666667, 262.78780625, 262.78770833333334, 262.78761041666667, 262.7875125, 262.7874145833333, 262.78731666666664, 262.78721875, 262.78712083333335, 262.7870229166667, 262.786925, 262.7868270833333, 262.78672916666665, 262.78663125, 262.7865333333333, 262.7864354166667, 262.7863375, 262.7862395833333, 262.78614166666665, 262.78604375, 262.78594583333336, 262.7858479166667, 262.78575, 262.78565208333333, 262.78555416666666, 262.78545625, 262.7853583333333, 262.7852604166667, 262.7851625]
2018-04-01 10:11:00: [262.78761041666667, 262.7875125, 262.7874145833333, 262.78731666666664, 262.78721875, 262.78712083333335, 262.7870229166667, 262.786925, 262.7868270833333, 262.78672916666665, 262.78663125, 262.7865333333333, 262.7864354166667, 262.7863375, 262.7862395833333, 262.78614166666665, 262.78604375, 262.78594583333336, 262.7858479166667, 262.78575, 262.78565208333333, 262.78555416666666, 262.78545625, 262.7853583333333, 262.7852604166667,

2018-01-01 06:18:00: [266.881675, 266.88168125, 266.8816875, 266.88169375, 266.8817, 266.88170625, 266.8817125, 266.88171875, 266.881725, 266.88173125, 266.8817375, 266.88174375, 266.88175, 266.88175625, 266.8817625, 266.88176875, 266.881775, 266.88178125, 266.8817875, 266.88179375, 266.8818, 266.88180625, 266.8818125, 266.88181875, 266.881825, 266.88183125, 266.8818375, 266.88184375, 266.88185, 266.88185625]
2018-01-01 06:00:00: [266.8815625, 266.88156875, 266.881575, 266.88158125, 266.8815875, 266.88159375, 266.8816, 266.88160625, 266.8816125, 266.88161875, 266.881625, 266.88163125, 266.8816375, 266.88164375, 266.88165, 266.88165625, 266.8816625, 266.88166875, 266.881675, 266.88168125, 266.8816875, 266.88169375, 266.8817, 266.88170625, 266.8817125, 266.88171875, 266.881725, 266.88173125, 266.8817375, 266.88174375]
2017-12-31 12:46:00: [266.8751, 266.87510625, 266.8751125, 266.87511875, 266.875125, 266.87513125, 266.8751375, 266.87514375, 266.87515, 266.87515625, 266.8751625, 266.8751

2017-12-24 04:30:00: [267.4860199833472, 267.4860074937552, 267.48599500416316, 267.48598251457116, 267.48597002497917, 267.4859575353872, 267.4859450457952, 267.4859325562032, 267.48592006661113, 267.48590757701913, 267.48589508742714, 267.48588259783514, 267.48587010824315, 267.4858576186511, 267.4858451290591, 267.4858326394671, 267.4858201498751, 267.4858076602831, 267.485795170691, 267.48578268109907, 267.48577019150713, 267.4857577019151, 267.48574521232314, 267.48573272273103, 267.48572023313903, 267.48570774354704, 267.48569525395504, 267.48568276436305, 267.485670274771, 267.485657785179]
2017-12-23 06:11:00: [267.50274354704413, 267.50273105745214, 267.5027185678601, 267.5027060782681, 267.5026935886761, 267.5026810990841, 267.5026686094921, 267.5026561199, 267.50264363030806, 267.5026311407161, 267.50261865112407, 267.50260616153207, 267.50259367194, 267.50258118234797, 267.50256869275603, 267.50255620316403, 267.50254371357204, 267.50253122398, 267.502518734388, 267.5025062

2017-12-14 07:58:00: [267.17, 267.17, 267.17, 267.17, 267.17, 267.17, 267.17, 267.17, 267.15, 267.135, 267.12, 267.1, 267.1, 267.09, 267.09, 267.09, 267.09, 267.105, 267.12, 267.15, 267.16, 267.17, 267.16333333333336, 267.15666666666664, 267.15, 267.15, 267.15, 267.16, 267.16, 267.1566666666667]
2017-12-13 19:22:00: [266.938, 266.936, 266.934, 266.932, 266.93, 266.93666666666667, 266.9433333333333, 266.95, 266.98, 266.975, 266.97, 266.97, 266.97, 266.99, 267.08, 267.1, 267.09, 267.08, 267.08, 267.0733333333333, 267.06666666666666, 267.06, 267.05, 267.07, 267.08, 267.09000000000003, 267.1, 267.1, 267.1, 267.11]
2017-12-13 12:08:00: [267.43, 267.39, 267.38, 267.4, 267.37, 267.3385, 267.37, 267.345, 267.33, 267.35, 267.33, 267.36, 267.54, 267.45, 267.42, 267.4385, 267.49, 267.52, 267.54, 267.525, 267.5, 267.499, 267.49, 267.39, 267.395, 267.22, 267.2, 267.1452, 267.17, 267.21]
2017-12-13 06:13:00: [266.72333333333336, 266.7266666666667, 266.73, 266.7325, 266.735, 266.7375, 266.74, 266.748

...

2018-08-18 18:04:00: [285.3257321428572, 285.3259375, 285.3261428571429, 285.3263482142857, 285.32655357142863, 285.32675892857145, 285.32696428571427, 285.32716964285714, 285.327375, 285.32758035714284, 285.3277857142857, 285.32799107142864, 285.32819642857135, 285.3284017857143, 285.32860714285715, 285.3288125, 285.32901785714284, 285.3292232142857, 285.32942857142865, 285.32963392857135, 285.3298392857143, 285.33004464285716, 285.33025, 285.33045535714285, 285.33066071428567, 285.3308660714285, 285.33107142857136, 285.3312767857143, 285.33148214285717, 285.3316875]
2018-08-19 05:51:00: [285.47091964285715, 285.4711250000001, 285.47133035714285, 285.4715357142857, 285.47174107142865, 285.47194642857136, 285.4721517857143, 285.47235714285716, 285.4725625, 285.47276785714286, 285.47297321428573, 285.4731785714285, 285.47338392857137, 285.4735892857143, 285.4737946428572, 285.474, 285.47420535714286, 285.47441071428574, 285.47461607142856, 285.47482142857143, 285.4750267857143, 285.4752

2018-08-17 08:56:00: [283.68, 283.63, 283.64, 283.66, 283.65, 283.68, 283.63, 283.64, 283.68, 283.71, 283.66, 283.71, 283.64, 283.66, 283.69, 283.71, 283.74, 283.73, 283.69, 283.73, 283.71, 283.72, 283.79, 283.8, 283.87, 283.9, 283.88, 283.9, 283.96, 284.01]
2018-08-17 08:45:00: [283.625, 283.64, 283.69, 283.71, 283.69, 283.64, 283.63, 283.66, 283.7, 283.74, 283.71, 283.68, 283.63, 283.64, 283.66, 283.65, 283.68, 283.63, 283.64, 283.68, 283.71, 283.66, 283.71, 283.64, 283.66, 283.69, 283.71, 283.74, 283.73, 283.69]
2018-08-17 07:53:00: [284.04, 284.07, 284.07, 284.066, 284.062, 284.058, 284.0540000000001, 284.05, 284.0, 283.95, 283.945, 283.94, 283.94, 284.0, 284.0, 283.99, 284.0, 283.97, 283.94, 283.92, 283.94, 283.92, 283.9, 283.92, 283.94, 283.9550000000001, 283.97, 283.97, 283.97, 283.97]
2018-08-17 07:45:00: [284.08, 284.04, 284.04, 284.035, 284.03, 283.95, 284.0, 284.02, 284.04, 284.07, 284.07, 284.066, 284.062, 284.058, 284.0540000000001, 284.05, 284.0, 283.95, 283.945, 283.94, 

2018-08-08 12:28:00: [285.49, 285.46, 285.43, 285.43, 285.385, 285.32, 285.35, 285.33, 285.355, 285.34, 285.34, 285.34, 285.31, 285.39, 285.36, 285.37, 285.345, 285.37, 285.4, 285.42, 285.43, 285.415, 285.43, 285.43, 285.42, 285.41, 285.3514, 285.29, 285.285, 285.2899]
2018-08-08 11:52:00: [285.63, 285.58, 285.53, 285.5, 285.5, 285.5199, 285.51, 285.58, 285.63, 285.69, 285.675, 285.72, 285.7201, 285.71, 285.69, 285.71, 285.7, 285.725, 285.75, 285.7290000000001, 285.7, 285.6922, 285.6636, 285.661, 285.6275, 285.57, 285.59, 285.56, 285.565, 285.519]
2018-08-08 11:23:00: [285.381, 285.39, 285.47, 285.52, 285.58, 285.57, 285.58, 285.58, 285.52, 285.565, 285.595, 285.62, 285.63, 285.639, 285.58, 285.56, 285.57, 285.55, 285.6, 285.58, 285.64, 285.55, 285.6127, 285.625, 285.65, 285.63, 285.6575, 285.63, 285.61, 285.63]
2018-08-08 11:18:00: [285.185, 285.18, 285.3, 285.335, 285.42, 285.381, 285.39, 285.47, 285.52, 285.58, 285.57, 285.58, 285.58, 285.52, 285.565, 285.595, 285.62, 285.63, 285.63

2018-03-09 06:00:00: [273.96, 273.94, 273.96000000000004, 273.98, 274.0, 273.99, 273.98, 273.98, 274.0, 274.02, 274.02666666666664, 274.03333333333336, 274.04, 274.08, 274.12, 274.18, 274.23, 274.24, 274.25, 274.26, 274.32, 274.325, 274.33, 274.33500000000004, 274.34000000000003, 274.345, 274.35, 274.3533333333333, 274.3566666666667, 274.36]
2018-03-09 04:31:00: [274.125, 274.12, 274.1, 274.11, 274.12, 274.1, 274.02, 274.0, 273.92, 273.91, 273.94500000000005, 273.98, 274.04, 274.08, 274.12, 274.12, 274.11, 274.1, 274.11, 274.105, 274.1, 274.11, 274.12, 274.13, 274.105, 274.08, 274.08, 274.08, 274.08, 274.06]
2018-03-08 08:30:00: [273.29, 273.26, 273.25, 273.27, 273.27, 273.33, 273.4, 273.4, 273.42, 273.36, 273.32, 273.32, 273.32, 273.31, 273.36, 273.35, 273.35, 273.44, 273.35, 273.36, 273.36, 273.28, 273.25, 273.28, 273.22, 273.26, 273.27, 273.28, 273.32, 273.3]
2018-08-01 04:49:00: [281.29, 281.25, 281.16, 281.16, 281.18333333333334, 281.2066666666667, 281.23, 281.26, 281.29, 281.29, 

KeyboardInterrupt: 

In [ ]:
merged

In [30]:
top_500 = merged.sort_values(by=['eod_price_diff_perc_magnitude'], ascending=False).iloc[:500, :]

KeyError: 'eod_price_diff_perc_magnitude'

In [ ]:
top_500

In [ ]:
top_500['cleaned_text'] = top_500['cleaned_text'].apply(lambda x: [item.lower() for item in x.split()])

In [ ]:
top_500.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

np.random.seed(2018)
import nltk
# nltk.download('wordnet')

# Dataset cleaning
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    text = re.sub(r"http\S+", "", text)
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

top_500['cleaned_text'] = top_500.loc[:,'cleaned_text'].apply(lambda x: preprocess(' '.join(x)))

In [ ]:
top_500.head()

In [ ]:
top_500_corpus = []

for this_list in top_500.loc[:, 'cleaned_text']:
    top_500_corpus.append(' '.join(this_list))

In [ ]:
top_500_corpus

In [ ]:
vectorizer = TfidfVectorizer(max_features=10, stop_words='english', max_df=0.06)
top_500_tfidf = vectorizer.fit_transform(top_500_corpus)
print(vectorizer.get_feature_names())

In [ ]:
#convert your matrix to an array to loop over it
mat_array = top_500_tfidf.toarray()

# get your feature names
fn = vectorizer.get_feature_names()

for l in mat_array: 
    print([(fn[x],l[x]) for x in (l*-1).argsort()][:10])